# Superbowl 2017


## tl;dr

Vamos a analizar una colección de tweets en inglés publicados durante un partido de fútbol.


## Contexto

El pasado 5 de febrero se celebró la [51ª edición de la Superbowl](https://en.wikipedia.org/wiki/Super_Bowl_LI), la gran final del campeonato de fútbol americano de la NFL. El partido enfrentó a los [New England Patriots](https://en.wikipedia.org/wiki/New_England_Patriots) (los favoritos, los de la costa este, con [Tom Brady](https://en.wikipedia.org/wiki/Tom_Brady) a la cabeza) contra los [Atlanta Falcons](https://en.wikipedia.org/wiki/Atlanta_Falcons) (los aspirantes, los del Sur, encabezados por [Matt Ryan](https://en.wikipedia.org/wiki/Matt_Ryan_(American_football))).

![](http://bandageek.com/wp-content/uploads/2017/02/patriots-vs-falcons.jpg)

Como cualquier final, el resultado *a priori* era impredecible y a un partido podía ganar cualquiera. Pero el del otro día fue un encuentro inolvidable porque comenzó con el equipo débil barriendo al favorito y con un Brady que no daba una. Al descanso, el marcador reflejaba un inesperado **3 - 28** y todo indicaba que los Falcons ganarían su primer anillo.

![](https://pbs.twimg.com/media/C38X-Z-VUAA-UAV.jpg)

Pero, en la segunda mitad, Brady resurgió... y su equipo comenzó a anotar una y otra vez... con los Falcons ko. Los Patriots consiguieron darle la vuelta al marcador y vencieron por **34 - 28** su quinta Superbowl. Brady fue elegido MVP del encuentro y aclamado como el mejor quaterback de la historia.

![](http://images.complex.com/complex/images/c_limit,w_680/f_auto,fl_lossy,pg_1,q_auto/d36dh2j3micwoszunssh/tom-brady-new-england-patriots-vince-lombardi-trophy-super-bowl-li)

Como os imaginaréis, tanto vaivén nos va a dar mucho juego a la hora de analizar un corpus de mensajes de Twitter. Durante la primera mitad, es previsible que encuentres mensajes a favor de Atlanta y burlas a New England y a sus jugadores, que no estaban muy finos. Pero al final del partido, con la remontada, las opiniones y las burlas cambiarán de sentido.

Como tanto Tom Brady como su entrenador, Bill Belichick, habían declarado públicamente sus preferencias por Donald Trump durante las elecciones a la presidencia, es muy probable que encuentres mensajes al respecto y menciones a demócratas y republicanos.

Por último, durante el *half time show* actuó Lady Gaga, que también levanta pasiones a su manera, así que es probable que haya menciones a otras *reinas* de la música y comparaciones con actuaciones pasadas.

![](http://www.billboard.com/files/styles/article_main_image/public/media/12-lady-gaga-super-bowl-feb-2017-billboard-1548.jpg)


## Los datos

El fichero `2017-superbowl-tweets.tsv` ubicado en el directorio `data/` contiene una muestra, ordenada cronológicamente, de mensajes escritos en inglés publicados antes, durante y después del partido. Todos los mensajes contienen el hashtag `#superbowl`. Hazte una copia de este fichero en el directorio `notebooks` de tu espacio personal.

El fichero es en realidad una tabla con cuatro columnas separadas por tabuladores, que contiene líneas (una por tweet) con el siguiente formato:

    id_del_tweet fecha_y_hora_de_publicación autor_del_tweet texto_del_mensaje


La siguiente celda te permite abrir el fichero para lectura y cargar los mensajes en la lista `tweets`. Modifica el código para que la ruta apunte a la copia local de tu fichero.

In [ ]:
!gunzip ../data/2017-twitter-messages.tsv.gz
!ls -l ../data

In [1]:
tweets = []
RUTA = '../data/2017-twitter-messages.tsv'
for line in open(RUTA).readlines():
    tweets.append(line.split('\t'))

In [ ]:
# in Windows, make sure you read the tweets in unicode
import codecs
tweets = []
RUTA = '../data/2017-twitter-messages.tsv'

with codecs.open(RUTA, "r", "utf-8") as f:
    for line in f.readlines():
        tweets.append(line.split("\t"))

Fíjate en la estructura de la lista: se trata de una lista de tuplas con cuatro elementos. Puedes comprobar si el fichero se ha cargado como debe en la siguiente celda:

In [2]:
ultimo_tweet = tweets[-1]
print('id =>', ultimo_tweet[0])
print('fecha =>', ultimo_tweet[1])
print('autor =>', ultimo_tweet[2])
print('texto =>', ultimo_tweet[3])

id => 828498211253997568
fecha => 2017-02-06 06:59:00
autor => ceebrie
texto => Honestly WHAT a SuperBowl performance ✨



## Al lío

A partir de aquí puedes hacer distintos tipos de análisis. Añade tantas celdas como necesites para intentar, por ejemplo:

- calcular distintas estadísticas de la colección: número de mensajes, longitud de los mensajes, presencia de hashtags y emojis, etc.
- número de menciones a usuarios, frecuencia de aparición de menciones, frecuencia de autores
- calcular estadísticas sobre usuarios: menciones, mensajes por usuario, etc.
- calcular estadísticas sobre las hashtags
- calcular estadísticas sobre las URLs presentes en los mensajes
- calcular estadísticas sobre los emojis y emoticonos de los mensajes
- extraer automáticamente las entidades nombradas que aparecen en los mensajes y su frecuencia
- procesar los mensajes para extraer y analizar opiniones: calcular la subjetividad y la polaridad de los mensajes
- extraer las entidades nombradas que levantan más pasiones, quiénes son los más queridos y los más odiados, atendiendo a la polaridad de los mensajes
- comprobar si la polaridad de alguna entidad varía radicalmente a medida que avanza el partido
- cualquier otra cosa que se te ocurra :-P



In [3]:
# escribe tu código aquí
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.models import TextClassifier

# cargamos el reconocedor de entidades
tagger = SequenceTagger.load("ner-fast")
classifier = TextClassifier.load("en-sentiment")

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
2019-04-13 13:18:25,172 loading file /home/victor/.flair/models/en-ner-fast-conll03-v0.2.pt
2019-04-13 13:18:27,296 loading file /home/victor/.flair/models/imdb.pt


/home/victor/.virtualenvs/ds/lib/python3.6/site-packages/torch/serialization.py:542: DeprecationWarning: Call to deprecated class DocumentLSTMEmbeddings. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  result = unpickler.load()


In [5]:
for tweet in tweets[200:400]:
    if len(tweet) == 4:
        sentence = Sentence(tweet[3])
        tagger.predict(sentence)

        # iteramos por la entidades
        for entity in sentence.get_spans("ner"):
            if entity.tag == "PER":
                classifier.predict(sentence)
                print(entity, sentence.labels)            

PER-span [3,4]: "Tom Brady's" [NEGATIVE (1.0)]
PER-span [13,14]: "#PepsiHalftime I've" [NEGATIVE (0.27189400792121887)]
PER-span [7]: "@Madonna" [POSITIVE (0.16003376245498657)]
PER-span [5]: "#RiseUp" [POSITIVE (1.0)]
PER-span [7]: "Rosner" [POSITIVE (0.4890175759792328)]
PER-span [6,7]: "Bruno Mars" [POSITIVE (1.0)]
PER-span [17,18]: "Kellyanne Conway" [NEGATIVE (0.21716229617595673)]
PER-span [19,20]: "Lady Gaga" [POSITIVE (0.17029157280921936)]
PER-span [3,4]: "Tom Brady's" [NEGATIVE (1.0)]
PER-span [8]: "@ladygaga" [NEGATIVE (0.7228256464004517)]
PER-span [10,11]: "Lady Gaga" [NEGATIVE (0.23199717700481415)]
PER-span [3]: "God" [POSITIVE (0.8185834884643555)]
PER-span [12]: "Satan" [POSITIVE (0.8185834884643555)]
PER-span [3,4]: "Tom Brady's" [POSITIVE (0.28234121203422546)]
PER-span [3,4]: "Donatella Versace" [POSITIVE (0.470921128988266)]
PER-span [11,12]: "Lady Gaga" [POSITIVE (0.470921128988266)]
PER-span [3,4]: "Paul Pierce" [POSITIVE (1.0)]
PER-span [4]: "Gaga" [POSITIVE (0.